# EMISSOR chat bot with a BRAIN that interacts through camera and NER

@CLTL

This notebook demonstrates a simple chatbot that chats with you and stores entity mentions both as annotations and in the BRAIN.

Requires spacy and a language model installed and available in the venv kernel of Jupyter!

In [1]:
# general imports for EMISSOR and the BRAIN

from datetime import datetime

import requests
import spacy
from cltl import brain
from cltl.reply_generation.lenka_replier import LenkaReplier

/Users/selbaez/Documents/PhD/CLTL/cltl-chatbots/venv-cltl-chatbots/lib/python3.7/site-packages/rdflib_jsonld/__init__.py:12: DeprecationWarning: The rdflib-jsonld package has been integrated into rdflib as of rdflib==6.0.1.  Please remove rdflib-jsonld from your project's dependencies.
  DeprecationWarning,


In [2]:
import sys
import os

src_path = os.path.abspath(os.path.join('..'))
if src_path not in sys.path:
    sys.path.append(src_path)

#### The next utils are needed for the interaction and creating triples and capsules
import util.driver_util as d_util
import util.capsule_util as c_util
import util.text_util as t_util
import dummies.text_to_triple as ttt

### Initialise spacy

If you have not done so, you need to download the specific language module from spaCy using the terminal within the same venv: (

`venv)(bas)% python -m spacy download en_core_web_sm`

In [3]:
### Load a language model in spaCy
nlp = spacy.load('en_core_web_sm')

### Initialise the BRAIN

Before we start, we need to create an empty Brain or load an existing Brain. The next code assumes we have a repository in GraphDB with the name sandbox as a brain. By setting clear_all=True it is emptied and next loaded with the background ontologies.

In [4]:
import pathlib

log_path = pathlib.Path('./logs')
print(type(log_path))
my_brain = brain.LongTermMemory(address="http://localhost:7200/repositories/sandbox",
                                log_dir=log_path,
                                clear_all=True)

<class 'pathlib.PosixPath'>
2021-11-03 16:24:53,488 -     INFO -    cltl.brain.basic_brain.LongTermMemory - Uploading ontology to brain
2021-11-03 16:24:55,271 -     INFO -   cltl.brain.basic_brain.TrustCalculator - Computed trust for all known agents


## Standard initialisation of a scenario

In [5]:
from random import getrandbits

replier = LenkaReplier()

##### Setting the location
place_id = getrandbits(8)
location = requests.get("https://ipinfo.io").json()

##### Setting the agents
AGENT = "Leolani2"
HUMAN_NAME = "Stranger"
HUMAN_ID = "stranger"

### The name of your scenario
scenario_id = datetime.today().strftime("%Y-%m-%d-%H:%M:%S")

### Specify the path to an existing data folder where your scenario is created and saved as a subfolder
scenario_path = "../../data"

### Define the folder where the images are saved
imagefolder = scenario_path + "/" + scenario_id + "/" + "image"

### Create the scenario folder, the json files and a scenarioStorage and scenario in memory
scenarioStorage = d_util.create_scenario(scenario_path, scenario_id)
scenario = scenarioStorage.create_scenario(scenario_id, datetime.now().microsecond, datetime.now().microsecond, AGENT)

2021-11-03 16:24:55,279 -    DEBUG -   cltl.reply_generation.api.LenkaReplier - Booted
Directory  ../../data/2021-11-03-16:24:55  Created 
Directory  ../../data/2021-11-03-16:24:55/image  Created 


## 4 Starting the interaction

In [ ]:
##### First signals to get started

#### Initial prompt by the system from which we create a TextSignal and store it
initial_prompt = "Hi there. Who are you " + HUMAN_NAME + "?"
print(AGENT + ": " + initial_prompt)
textSignal = d_util.create_text_signal(scenario, initial_prompt)
scenario.append_signal(textSignal)

utterance = input('\n')
print(HUMAN_NAME + ": " + utterance)
while not (utterance.lower() == 'stop' or utterance.lower() == 'bye'):
    textSignal = d_util.create_text_signal(scenario, utterance)

    ### Apply some processing to the textSignal and add annotations
    entityText = t_util.add_ner_annotation_with_spacy(textSignal, nlp)
    scenario.append_signal(textSignal)

    ## Post triples to the brain:
    ### we use a dummy function that creates a denotedBy triple for the entityText
    subj, pred, obj = ttt.getTriplesFromEntities(entityText, textSignal.id)

    response = {}
    if not subj == "":
        print('Subject:', subj, 'Predicate:', pred, 'Object:', obj)
        perspective = {"certainty": 1, "polarity": 1, "sentiment": 1}
        capsule = c_util.scenario_utterance_to_capsule_with_perspective(scenario, place_id, location, textSignal,
                                                                        HUMAN_ID, perspective, subj, pred, obj)

        print('Capsule:', capsule)

        response = my_brain.update(capsule, reason_types=True,
                                   create_label=False)  ## use this version if you want to use the URI as subject
        #response = my_brain.update(capsule, reason_types=True) ## this function creates a label from the subject

        #print(thoughts)

    # Create the response from the system and store this as a new signal
    utterance = ttt.getTextFromTriples(response)
    if not utterance:
        if not entityText:
            utterance = "Any gossip" + '\n'
        else:
            utterance = "So you what do you want to talk about " + entityText[0] + '\n'

    response = utterance[::-1]
    print(AGENT + ": " + utterance)
    textSignal = d_util.create_text_signal(scenario, utterance)
    scenario.append_signal(textSignal)

    ###### Getting the next input signals
    utterance = input('\n')

Leolani2: Hi there. Who are you Stranger?



 I am from Amsterdam


Stranger: I am from Amsterdam
Subject: Amsterdam Predicate: denotedBy Object: a6449c86-d9be-4673-bb4f-c787c6b20e37
Capsule: {'chat': '2021-11-03-16:24:55', 'turn': 'a6449c86-d9be-4673-bb4f-c787c6b20e37', 'author': 'stranger', 'utterance': 'I am from Amsterdam', 'utterance_type': <UtteranceType.STATEMENT: 0>, 'position': '0-19', 'subject': {'label': 'Amsterdam', 'type': 'person'}, 'predicate': {'type': 'denotedBy'}, 'object': {'label': 'a6449c86-d9be-4673-bb4f-c787c6b20e37', 'type': 'object'}, 'perspective': {'certainty': 1, 'polarity': 1, 'sentiment': 1}, 'context_id': 'Leolani2', 'date': datetime.date(2021, 11, 3), 'place': 'Amsterdam', 'place_id': 81, 'country': 'NL', 'region': 'North Holland', 'city': 'Amsterdam', 'objects': [{'type': 'chair', 'confidence': 0.59, 'id': 1}, {'type': 'table', 'confidence': 0.73, 'id': 1}, {'type': 'pillbox', 'confidence': 0.32, 'id': 1}], 'people': [{'name': 'stranger', 'confidence': 0.98, 'id': 1}]}
2021-11-03 16:25:09,969 -     INFO -    cltl.brain.

### After we stopped the interaction, we set the end time of the scenarion

In [ ]:
## scenario.scenario.end = datetime.now().microsecond

### 4.4 Saving the Scenario

In [ ]:
scenarioStorage.save_scenario(scenario)

## End of notebook